**用于将GEE中LandTrendr处理后的拟合影像分为独立波段数据**

例如：

获取的52年的[NDVI, TCG, TCW]数据得到为52x3

本代码将数据分割为独立3波段的数据，51x1 for NDVI/TCG/TCW

In [ ]:
!pip install rasterio

In [11]:
import rasterio
import numpy as np

def split_bands(img_path, win_span,st_yr):
    # 打开影像
    with rasterio.open(img_path) as src:
        # 获取波段数量
        bands_count = src.count
        # 计算需要分割的次数
        splits_count = bands_count // win_span + (1 if bands_count % win_span != 0 else 0)

        # 分割波段
        for i in range(splits_count):
            # 计算当前分割的起始和结束波段索引
            start_band = i * win_span
            end_band = min((i + 1) * win_span, bands_count)
            print(start_band,end_band)

            # 读取当前分割的波段
            bands = src.read(range(start_band + 1, end_band + 1))

            # 创建新的波段名称
            new_band_names = [f"ndvi{st_yr + j}" if j < win_span // 3 else f"tcg{st_yr + j - win_span // 3}" if j < 2 * win_span // 3 else f"tcw{st_yr + j - 2 * win_span // 3}" for j in range(end_band - start_band)]

            # 创建新的影像文件
            new_img_path = f"{img_path.split('.')[0]}_split_{i}.tif"
            with rasterio.open(new_img_path, 'w', driver='GTiff', height=src.height, width=src.width, count=end_band - start_band, dtype=bands.dtype, crs=src.crs, transform=src.transform) as dst:
                # 写入波段数据
                for j, band in enumerate(bands):
                    dst.write(band, j + 1)
                    # 设置波段名称
                    dst.set_band_description(j + 1, new_band_names[j])


# 使用示例
img_path = '/content/eetile2x15-5-NDVI-11-19722023-06150915.tif'
win_span = 52
split_bands(img_path, win_span)

0 52
52 104
104 156
156 157
